# Notebook for experiment tracking with Weight and Biases 

In [1]:
import matplotlib.pyplot as plt

# from gridworlds.grid_env import GridEnvironment
from src.Generalist.generalist_meta_env import Generalist_MetaEpisodeEnv
from src.Generalist.draw_gridworld import draw_policy

# import gymnasium as gym
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed

# wandb 
import wandb
from wandb.integration.sb3 import WandbCallback

#stablebaselines feature extractor
from src.Generalist.feature_extractor import Custom_Flatten

#For evaluation
from src.Generalist.evals_utils import average_evals

# Load Gridworlds

In [2]:
#Load the gridworlds
from classes import Object
import pickle

with open('src/world_builder/worlds/master_set_train.pkl','rb') as f:
    train_gridworlds = pickle.load(f)
print(f'{len(train_gridworlds)} train gridworlds loaded')  
for grid in train_gridworlds:
    grid.early_stopping = False   

with open('src/world_builder/worlds/master_set_val.pkl','rb') as f:
    val_gridworlds = pickle.load(f)
print(f'{len(val_gridworlds)} val gridworlds loaded')  
for grid in val_gridworlds:
    grid.early_stopping = False   

with open('src/world_builder/worlds/master_set_test.pkl','rb') as f:
    test_gridworlds = pickle.load(f)
print(f'{len(test_gridworlds)} test gridworlds loaded')  
for grid in test_gridworlds:
    grid.early_stopping = False        

976 train gridworlds loaded
96 val gridworlds loaded
200 test gridworlds loaded


# Training Cell

In [ ]:
sweep_config = {
    "method": "grid",
    "metric": {"goal": "maximize", "name": "train_metrics/Usefulness"},
    "parameters": {
        "lambda_factor": {"value": 0.9},
        "meta_ep_size": {"value": 32},
        "hidden_layer_depth": {'value': 128},
        "num_hidden_layers": {'value': 3},
        "ent_coef": {'value': 0.015},
        "learning_rate": {'value': 0.0000005},
        "total_timesteps": {'value': 20000000},
        "clip_range": {'value': 0.2},
        "n_steps_ppo": {'value': 8192},
        "batch_size": {'value': 64},
        "vf_coef": {'values': [0.5,0.55,0.6]},
        "timesteps_per_run": {'value': 20000000}
    },
}

In [33]:
sweep_id = wandb.sweep(sweep_config, project="IPP-second-paper-generalist")
print(sweep_id)

Create sweep with ID: 7a7xk29f
Sweep URL: https://wandb.ai/IPP-experiments/IPP-second-paper-generalist/sweeps/7a7xk29f
7a7xk29f


In [16]:
## BEFORE RUNNING, MAKE SURE TO MANUALLY CHANGE ENVIRONMENT LIST AND FEATURE EXTRACTOR ##

def train(config=None):
    run = wandb.init(config=config)
    config = wandb.config

    ## PICK ENVIRONMENT
    train_env_list = train_gridworlds           #MANUALLY CHANGE
    test_env_list = val_gridworlds              #MANUALLY CHANGE

    hld = config.hidden_layer_depth
    num_layers = config.num_hidden_layers

    def net_arch(hidden_layer_depth, num_hidden_layers):
        net_arch_list = []
        for n in range(num_hidden_layers):
            net_arch_list.append(hidden_layer_depth)
        return net_arch_list      

    net_arch_list = net_arch(hld, num_layers)

    policy_kwargs = dict(features_extractor_class=Custom_Flatten, #MANUALLY CHANGE
                        features_extractor_kwargs=dict(features_dim=250),
                        net_arch=dict(pi=net_arch_list, 
                                    vf=net_arch_list))

    #Number of vectorised environments
    num_cpu=3

    #Set-up for vectorised environments
    def make_env(rank, seed=0):
            """
            Utility function for multiprocessed env.

            :param env_id: (str) the environment ID
            :param seed: (int) the inital seed for RNG
            :param rank: (int) index of the subprocess
            """

            def _init():
                env = Generalist_MetaEpisodeEnv(
                        train_env_list, 
                        meta_ep_size=config.meta_ep_size,
                        lambda_factor=config.lambda_factor,
                    )
                # use a seed for reproducibility
                # Important: use a different seed for each environment
                # otherwise they would generate the same experiences
                env.reset(seed=seed + rank)
                return env

            set_random_seed(seed)
            return _init


    def vec_learning_run(model, timesteps):

        env = SubprocVecEnv([make_env(i) for i in range(num_cpu)],start_method="fork")

        model.set_env(env)

        model.learn(total_timesteps=timesteps,
                    callback=WandbCallback(verbose=0)) 

        return model


    def vec_learning(train_env_list,timesteps_per_run, total_timesteps):

        wandb.define_metric("custom_step")

        # Define which metrics to plot against that x-axis
        wandb.define_metric("train_metrics/Usefulness", step_metric='custom_step')
        wandb.define_metric("train_metrics/Neutrality", step_metric='custom_step')

        steps_count = 0

        env = SubprocVecEnv([make_env(i) for i in range(num_cpu)],start_method="fork")

        # Create the PPO model with the custom architecture
        model = PPO("MlpPolicy",                                  #MAUALLY CHANGE with feature_extractor_class
                    env,                                      #Change for vectorised Envs
                    verbose=1,
                    ent_coef=config.ent_coef,
                    learning_rate= config.learning_rate,
                    clip_range=config.clip_range,
                    n_steps=config.n_steps_ppo,
                    batch_size=config.batch_size,
                    vf_coef=config.vf_coef,
                    policy_kwargs=policy_kwargs,           #MANUALLY CHANGE
                    tensorboard_log=f"runs/{run.id}")
        
        while steps_count < total_timesteps:

            model = vec_learning_run(model, timesteps_per_run)
            train_av_traj_ratio, train_av_usefulness, train_av_entropy = average_evals(train_env_list,model)
            steps_count += timesteps_per_run
            print(f'Step count: {steps_count}')
            print(f'Average Usefulness: {train_av_usefulness}')
            print(f'Average NEUTRALITY: {train_av_entropy}')
            print(f'Average Trajectory Ratio: {train_av_traj_ratio}')


            wandb.log({
                "custom_step": steps_count,
                "train_metrics/Usefulness": train_av_usefulness,
                "train_metrics/Neutrality": train_av_entropy,
                        })

        return model, train_av_usefulness, train_av_entropy

    model, train_av_usefulness, train_av_entropy = vec_learning(train_env_list, config.timesteps_per_run, config.total_timesteps)

    model.save(f"models/{run.id}")

    print('Average evals for train data')
    print(f'Average USEFULNESS:{train_av_usefulness}')
    print(f'Average NEUTRALITY:{train_av_entropy}')
    print('\n')

    run.summary["train_av_usefulness"]=train_av_usefulness
    run.summary["train_av_NEUTRALITY"]=train_av_entropy

    test_av_traj, test_av_usefulness, test_av_entropy = average_evals(test_env_list,model)

    print('Average evals for test data')
    print(f'Average Trajectory Ratio:{"{:.3f}".format(test_av_traj)}')
    print(f'Average USEFULNESS:{test_av_usefulness}')
    print(f'Average NEUTRALITY:{test_av_entropy}')
    print('\n')

    run.summary["test_av_usefulness"]=test_av_usefulness
    run.summary["test_av_NEUTRALITY"]=test_av_entropy


In [34]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 53826vuu with config:
wandb: 	batch_size: 64
wandb: 	clip_range: 0.2
wandb: 	ent_coef: 0.015
wandb: 	hidden_layer_depth: 128
wandb: 	lambda_factor: 1
wandb: 	learning_rate: 5e-07
wandb: 	meta_ep_size: 1
wandb: 	n_steps_ppo: 8192
wandb: 	num_hidden_layers: 3
wandb: 	timesteps_per_run: 20000000
wandb: 	total_timesteps: 20000000
wandb: 	vf_coef: 0.5


Using cpu device
Logging to runs/53826vuu/PPO_1
------------------------------
| time/              |       |
|    fps             | 6066  |
|    iterations      | 1     |
|    time_elapsed    | 4     |
|    total_timesteps | 24576 |
------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 3697          |
|    iterations           | 2             |
|    time_elapsed         | 13            |
|    total_timesteps      | 49152         |
| train/                  |               |
|    approx_kl            | 3.1004092e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.39         |
|    explained_variance   | -0.12877607   |
|    learning_rate        | 5e-07         |
|    loss                 | 1.28          |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000133     |
|    value_loss           | 2.9

/Users/carissacullen/IPP/src/Generalist/evals_utils.py:285: RuntimeWarning: invalid value encountered in log2
  return np.log2(p)


Step count: 20000000
Average Usefulness: 0.9357349493387196
Average NEUTRALITY: nan
Average Trajectory Ratio: 2396192333676.7593
Average evals for train data
Average USEFULNESS:0.9357349493387196
Average NEUTRALITY:nan


Average evals for test data
Average Trajectory Ratio:72893.853
Average USEFULNESS:0.7272559714833137
Average NEUTRALITY:nan




custom_step,▁
train_metrics/Usefulness,▁
custom_step,20000000
test_av_usefulness,0.72726
train_av_usefulness,0.93573
train_metrics/Neutrality,nan
train_metrics/Usefulness,0.93573


...

Using cpu device
Logging to runs/81afhkqz/PPO_1
------------------------------
| time/              |       |
|    fps             | 6266  |
|    iterations      | 1     |
|    time_elapsed    | 3     |
|    total_timesteps | 24576 |
------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 3172          |
|    iterations           | 2             |
|    time_elapsed         | 15            |
|    total_timesteps      | 49152         |
| train/                  |               |
|    approx_kl            | 4.2288084e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.39         |
|    explained_variance   | -0.11766338   |
|    learning_rate        | 5e-07         |
|    loss                 | 2.04          |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000152     |
|    value_loss           | 3.0

/Users/carissacullen/IPP/src/Generalist/evals_utils.py:285: RuntimeWarning: invalid value encountered in log2
  return np.log2(p)


Step count: 20000000
Average Usefulness: 0.9407296066684497
Average NEUTRALITY: nan
Average Trajectory Ratio: 51534851269618.83
Average evals for train data
Average USEFULNESS:0.9407296066684497
Average NEUTRALITY:nan


Average evals for test data
Average Trajectory Ratio:85786.051
Average USEFULNESS:0.7122415787924261
Average NEUTRALITY:nan




custom_step,▁
train_metrics/Usefulness,▁
custom_step,20000000
test_av_usefulness,0.71224
train_av_usefulness,0.94073
train_metrics/Neutrality,nan
train_metrics/Usefulness,0.94073


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [19]:
from src.Generalist.evals_utils import evaluate_agent

In [21]:
model = PPO.load("models/yg7nv6rj")

In [24]:
traj_list = []
usefulness_list = []
entropy_list = []
grids=train_gridworlds
index_list = list(range(0,len(grids)))
for i in index_list:
    env = grids[i]
    trajectory_length_probs, usefulness, entropy = evaluate_agent(env, model, (env.max_coins[1],env.max_coins[0]))
    traj = [trajectory_length_probs[1],trajectory_length_probs[0]]
    traj_list.append(traj)
    usefulness_list.append(usefulness)
    entropy_list.append(entropy)
    if usefulness > 0.99:
        print(f"Gridworld: {i}")
        print(f"Usefulness: {usefulness}")
        print(f"Neutrality: {entropy}")

usefulness_av = sum(usefulness_list)/len(usefulness_list)
neutrality_av = sum(entropy_list)/len(entropy_list)
print(f'Average Usefulness: {usefulness_av}')
print(f'Average Neutrality: {neutrality_av}')    

Gridworld: 76
Usefulness: 0.9992817902548036
Neutrality: 0.0031877529471883866
Gridworld: 78
Usefulness: 0.9904011496798403
Neutrality: 0.07876977072162794
Gridworld: 80
Usefulness: 0.9992131731367985
Neutrality: 0.0044296500595050895
Gridworld: 81
Usefulness: 0.9991659836824981
Neutrality: 0.006982281720483381
Gridworld: 82
Usefulness: 0.9994492032625973
Neutrality: 0.0028219208432305643
Gridworld: 83
Usefulness: 0.9991647491571244
Neutrality: 0.005424695977139499
Gridworld: 84
Usefulness: 0.9994791009392769
Neutrality: 0.0036141521587449775
Gridworld: 85
Usefulness: 0.9983765858835564
Neutrality: 0.005649486444558387
Gridworld: 86
Usefulness: 0.9987894474241834
Neutrality: 0.004167686031657259
Gridworld: 87
Usefulness: 0.9990153140316844
Neutrality: 0.009057370570808567
Gridworld: 88
Usefulness: 0.9964117976350597
Neutrality: 0.03251868139669426
Gridworld: 89
Usefulness: 0.9994895838708029
Neutrality: 0.001991024713377191
Gridworld: 90
Usefulness: 0.9997512678865955
Neutrality: 0.001